In [5]:
from sqlalchemy import create_engine, MetaData, and_, or_, desc
from sqlalchemy import Table, Column, Integer, String, ForeignKey
from sqlalchemy.sql import text, select, insert, update, delete, func

#### Creer le chemin vers le document 

In [6]:
path = "sqlite:////Users/digitalisdiana/Downloads/chinook.db"

#### Creation de la variable

In [7]:
engine = create_engine(path, echo = False)

In [8]:
engine

Engine(sqlite:////Users/digitalisdiana/Downloads/chinook.db)

#### Context Manager : Python

In [9]:
with engine.connect() as connection:
    query = text("SELECT * FROM Albums WHERE AlbumId = 5")
    
    #requete de stockage
    resultset = connection.execute(query)
    
    #recuperation d'un resultat
    data = resultset.fetchone()
    print(f"Result: {data}")

Result: (5, 'Big Ones', 3)


In [10]:
with engine.connect() as connection:
    #Requete parametree {with albumid as parametre}
    query = text("SELECT * FROM Albums WHERE AlbumId = :AlbumId")
    
     #requete de stockage
    resultset = connection.execute(query, AlbumId = 5)
    data = resultset.fetchone()
    print(f"Result: {data}")

Result: (5, 'Big Ones', 3)


In [11]:
with engine.connect() as connection:
    #Requete parametree {with albumid as parametre}
    query = text("SELECT * FROM Albums WHERE AlbumId = :AlbumId")
    
     #requete de stockage
    for AlbumId in range (1,19):
        resultset = connection.execute(query, AlbumId = AlbumId)
        data = resultset.fetchone()
        print(f"Result: {data}")

Exception closing connection <sqlite3.Connection object at 0x10a21bab0>
Traceback (most recent call last):
  File "/Users/digitalisdiana/anaconda3/lib/python3.6/site-packages/sqlalchemy/pool.py", line 317, in _close_connection
    self._dialect.do_close(connection)
  File "/Users/digitalisdiana/anaconda3/lib/python3.6/site-packages/sqlalchemy/engine/default.py", line 463, in do_close
    dbapi_connection.close()
sqlite3.OperationalError: unable to close due to unfinalized statements or unfinished backups


Result: (1, 'For Those About To Rock We Salute You', 1)
Result: (2, 'Balls to the Wall', 2)
Result: (3, 'Restless and Wild', 2)
Result: (4, 'Let There Be Rock', 1)
Result: (5, 'Big Ones', 3)
Result: (6, 'Jagged Little Pill', 4)
Result: (7, 'Facelift', 5)
Result: (8, 'Warner 25 Anos', 6)
Result: (9, 'Plays Metallica By Four Cellos', 7)
Result: (10, 'Audioslave', 8)
Result: (11, 'Out Of Exile', 8)
Result: (12, 'BackBeat Soundtrack', 9)
Result: (13, 'The Best Of Billy Cobham', 10)
Result: (14, 'Alcohol Fueled Brewtality Live! [Disc 1]', 11)
Result: (15, 'Alcohol Fueled Brewtality Live! [Disc 2]', 11)
Result: (16, 'Black Sabbath', 12)
Result: (17, 'Black Sabbath Vol. 4 (Remaster)', 12)
Result: (18, 'Body Count', 13)


In [12]:
def rec_data(Id):
    
    with engine.connect() as connection:
        #Requete parametree {with albumid as parametre}
        query = text("SELECT * FROM Albums WHERE AlbumId = :AlbumId")

         #requete de stockage
        resultset = connection.execute(query, AlbumId = Id)
        data = resultset.fetchone()
        print(f"Result: {data}")

In [13]:
rec_data(3)

Result: (3, 'Restless and Wild', 2)


In [14]:
with engine.connect() as connection:
    #Requete parametree {with albumid as parametre}
    query = text("INSERT INTO Artists(Name) VALUES (:Name)")
    
     #requete de stockage
    resultset = connection.execute(query, Name = "Foobar")
    print(resultset.rowcount)

1


In [15]:
with engine.connect() as connection:
    query = text("UPDATE Artists SET Name = :Name WHERE ArtistId = :ArtistId")
    resultset = connection.execute(query, Name = "Baz", ArtistId = 3)
    print(resultset.rowcount)

1


# New way of doing things

In [16]:
metadata = MetaData()
metadata.bind = engine
metadata.reflect()

In [17]:
employees = Table("employees", metadata, autoload = True)

In [18]:
with engine.connect() as connection:
    query = select([employees]).limit(5)
    resultset = connection.execute(query)
    data = resultset.fetchall()
    for l in data:
        print(f"{l}")

(1, 'Adams', 'Andrew', 'General Manager', None, datetime.datetime(1962, 2, 18, 0, 0), datetime.datetime(2002, 8, 14, 0, 0), '11120 Jasper Ave NW', 'Edmonton', 'AB', 'Canada', 'T5K 2N1', '+1 (780) 428-9482', '+1 (780) 428-3457', 'andrew@chinookcorp.com')
(2, 'Edwards', 'Nancy', 'Sales Manager', 1, datetime.datetime(1958, 12, 8, 0, 0), datetime.datetime(2002, 5, 1, 0, 0), '825 8 Ave SW', 'Calgary', 'AB', 'Canada', 'T2P 2T3', '+1 (403) 262-3443', '+1 (403) 262-3322', 'nancy@chinookcorp.com')
(3, 'Peacock', 'Jane', 'Sales Support Agent', 2, datetime.datetime(1973, 8, 29, 0, 0), datetime.datetime(2002, 4, 1, 0, 0), '1111 6 Ave SW', 'Calgary', 'AB', 'Canada', 'T2P 5M5', '+1 (403) 262-3443', '+1 (403) 262-6712', 'jane@chinookcorp.com')
(4, 'Park', 'Margaret', 'Sales Support Agent', 2, datetime.datetime(1947, 9, 19, 0, 0), datetime.datetime(2003, 5, 3, 0, 0), '683 10 Street SW', 'Calgary', 'AB', 'Canada', 'T2P 5G3', '+1 (403) 263-4423', '+1 (403) 263-4289', 'margaret@chinookcorp.com')
(5, 'Joh

In [19]:
with engine.connect() as connection:
    columns = [employees.c.FirstName, employees.c.LastName]
    query = select(columns)
    query = query.where(employees.c.FirstName.like("%o%"))
    resultset = connection.execute(query)
    data = resultset.fetchall()
    for l in data:
        print(f"{l}")

('Robert', 'King')


In [21]:
with engine.connect() as connection:
    columns = [employees.c.FirstName, employees.c.LastName]
    query = select(columns)
    query = query.where(employees.c.EmployeeId==5)
    resultset = connection.execute(query)
    data = resultset.fetchall()
    for l in data:
        print(f"{l}")

('Steve', 'Johnson')


In [23]:
with engine.connect() as connection:
    columns = [employees.c.FirstName, employees.c.LastName]
    query = select(columns) 
    query = query.where(employees.c.FirstName.like("%o%")) 
    resultset = connection.execute(query)
    data = resultset.fetchall()
    for l in data:
        print(f"{l}")

('Robert', 'King')


In [25]:
with engine.connect() as connection:
    columns = [employees.c.FirstName, employees.c.LastName]
    #query = select(columns) 
    query = select(columns).where(employees.c.FirstName.like("%o%")) 
    resultset = connection.execute(query)
    data = resultset.fetchall()
    for l in data:
        print(f"{l}")

('Robert', 'King')


In [27]:
with engine.connect() as connection:
   #columns = [employees.c.FirstName, employees.c.LastName]
    #clauses = or_(employees.c.FirstName.in_(["Andrew", "Nancy"]), employees.c.PostalCode.like("%5%"))    
    #query = select([employees.c.FirstName, employees.c.LastName])
    query = select([employees.c.FirstName, employees.c.LastName]).where(or_(employees.c.FirstName.in_(["Andrew", "Nancy"]), employees.c.PostalCode.like("%5%")))
    resultset = connection.execute(query)
    data = resultset.fetchall()
    for l in data:
        print(f"{l}")

('Andrew', 'Adams')
('Nancy', 'Edwards')
('Jane', 'Peacock')
('Margaret', 'Park')
('Michael', 'Mitchell')
('Robert', 'King')


In [28]:
with engine.connect() as connection:
    #columns = [employees.c.FirstName, employees.c.LastName]
    #clauses = or_(employees.c.FirstName.in_(["Andrew", "Nancy"]), employees.c.PostalCode.like("%5%"))    
    #query = select([employees.c.FirstName, employees.c.LastName])
    #query = select([employees.c.FirstName, employees.c.LastName]).where(or_(employees.c.FirstName.in_(["Andrew", "Nancy"]), employees.c.PostalCode.like("%5%")))    
    #query = select([employees.c.FirstName, employees.c.LastName]).where(or_(employees.c.FirstName.in_(["Andrew", "Nancy"]), employees.c.PostalCode.like("%5%"))).order_by(desc(employees.c.EmployeeId))    
    #resultset = connection.execute(select([employees.c.FirstName, employees.c.LastName]).where(or_(employees.c.FirstName.in_(["Andrew", "Nancy"]), employees.c.PostalCode.like("%5%"))).order_by(desc(employees.c.EmployeeId)))
    data = connection.execute(select([employees.c.FirstName, employees.c.LastName]).where(or_(employees.c.FirstName.in_(["Andrew", "Nancy"]), employees.c.PostalCode.like("%5%"))).order_by(desc(employees.c.EmployeeId))).fetchall()
    for l in data:
        print(f"{l}")

('Robert', 'King')
('Michael', 'Mitchell')
('Margaret', 'Park')
('Jane', 'Peacock')
('Nancy', 'Edwards')
('Andrew', 'Adams')


In [40]:
#UPDATE Command
with engine.connect() as connection:
    query = update(employees).values(LastName = "Samuels").where(employees.c.EmployeeId == 2)
    connection.execute(query)
#DELETE Command
with engine.connect() as connection:
    query = delete(employees)
    query = query.where(employees.c.LastName.like("%ar"))
    results = connection.execute(query)

In [37]:
#INSERT Command
with engine.connect() as connection:
    query = insert(employees).values(FirstName='foo', LastName="bar")
    connection.execute(query)

###### Verification

In [39]:
#INSERT Command
with engine.connect() as connection:
    columns = [employees.c.EmployeeId, employees.c.FirstName, employees.c.LastName]
    query = select(columns).where(employees.c.FirstName == "foo" and employees.c.LastName == "bar")
    resultset = connection.execute(query)
    data = resultset.fetchall()
    print(data)

[(13, 'foo', 'bar')]
